In [1]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
class CNNBlock(layers.Layer):
  def __init__(self, out_channels, kernel_size = 3):
    super(CNNBlock, self).__init__()
    self.conv = layers.Conv2D(out_channels, kernel_size, padding = "same")
    self.bn = layers.BatchNormalization()

  def call(self, input_tensor, training = False):
    x = self.conv(input_tensor)
    x = self.bn(x, training = training)
    x = tf.nn.relu(x)
    return x

In [8]:
model = keras.Sequential([
    CNNBlock(32),
    CNNBlock(64),
    CNNBlock(128),
    layers.Flatten(),
    layers.Dense(10),
])

In [11]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(),
    metrics = ["accuracy"],
)

In [12]:
model.fit(
    x_train, y_train, batch_size = 64, epochs = 5
)

Epoch 1/5
938/938 [==============================] - 21s 13ms/step - loss: 0.5102 - accuracy: 0.9481
Epoch 2/5
938/938 [==============================] - 13s 14ms/step - loss: 0.0937 - accuracy: 0.9813
Epoch 3/5
938/938 [==============================] - 13s 14ms/step - loss: 0.0331 - accuracy: 0.9902
Epoch 4/5
938/938 [==============================] - 12s 12ms/step - loss: 0.0259 - accuracy: 0.9911
Epoch 5/5
938/938 [==============================] - 12s 12ms/step - loss: 0.0251 - accuracy: 0.9918


In [14]:
class ResBlock(layers.Layer):
  def __init__(self, channels):
    super(ResBlock, self).__init__()
    self.cnn1 = CNNBlock(channels[0])
    self.cnn2 = CNNBlock(channels[1])
    self.cnn3 = CNNBlock(CNNBlock[2])
    self.pooling = layers.MaxPooling2D()
    self.identity_mapping = layers.Conv2D(channels[1], 1, padding = "same")

  def call(self, input_tensor, training = False):
    x = self.cnn1(input_tensor, training = training)
    x = self.cnn2(x, training = training)
    x = self.cnn3(
        x + self.identity_mapping(input_tensor), training = training)
    
    return self.pooling(x)

In [15]:
class ResNet_Like(keras.Model):
  def __init__(self, num_classes = 10):
    super(ResNet_Like, self).__init__()
    self.block1 = ResBlock([32, 32, 64])
    self.block2 = ResBlock([128, 128, 256])
    self.block3 = ResBlock([128, 256, 512])
    self.pool = layers.GlobalAveragePooling2D()
    self.classifier = layers.Dense(num_classes)

  def call(self, input_tensor, training = False):
    x = self.block1(input_tensor, training = training)
    x = self.block2(x, training = training)
    x = self.block3(x, training = training)
    x = self.pool(x)
    return self.classifier(x)

In [16]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(),
    metrics = ["accuracy"],
)

In [ ]:
model.fit(
    x_train, y_train, batch_size = 64, epochs = 10
)

Epoch 1/10
938/938 [==============================] - 12s 12ms/step - loss: 0.0334 - accuracy: 0.9894
Epoch 2/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0172 - accuracy: 0.9947
Epoch 3/10
938/938 [==============================] - 12s 13ms/step - loss: 0.0145 - accuracy: 0.9954
Epoch 4/10
938/938 [==============================] - 12s 12ms/step - loss: 0.0109 - accuracy: 0.9965
Epoch 5/10
938/938 [==============================] - 12s 12ms/step - loss: 0.0088 - accuracy: 0.9972
Epoch 6/10
938/938 [==============================] - 12s 12ms/step - loss: 0.0064 - accuracy: 0.9979
Epoch 7/10
502/938 [===============>..............] - ETA: 5s - loss: 0.0064 - accuracy: 0.9980

In [ ]:
model.evaluate(
    x_test, y_test, batch_size = 64
)

In [ ]:
model.summary()